3.-Divide aleatoriamente el conjunto de datos de cáncer de seno de Wisconsin3

en un subconjunto de
entrenamiento con el 60 % de los datos, un subconjunto de validación con el 20 % y un subconjunto
de prueba con el 20 % restante usando 0 como semilla para tu generador de números aleatorios.
Este conjunto de datos contiene 699 registros de tumores de seno, de los cuales 458 son benignos y
241 son malignos.

In [1]:
#Librerias
import pandas as pd
from sklearn.model_selection import train_test_split


In [2]:

# URL del archivo de datos
url = "http://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/breast-cancer-wisconsin.data"

# Cargar los datos en un DataFrame
df = pd.read_csv(url, header=None)  # No tiene encabezados, por eso usamos header=None


In [3]:
# Definir nombres de columnas en español
column_names = [
    "ID_Muestra", "Grosor_Tumor", "Uniformidad_Tamaño_Celular", 
    "Uniformidad_Forma_Celular", "Adhesión_Marginal", "Tamaño_Célula_Epitelial", 
    "Núcleos_Desnudos", "Cromatina_Blanda", "Nucléolos_Normales", 
    "Mitosis_Celular", "Clase"
]

# Asignar los nombres de columna
df.columns = column_names

In [4]:
#Validamos si coincide con la información proporcionada
pivot_table = df.pivot_table(values="ID_Muestra", index="Clase", aggfunc="count")
pivot_table.loc["Total"] = pivot_table.sum()
pivot_table["Porcentaje"] = (pivot_table["ID_Muestra"] / pivot_table.loc["Total", "ID_Muestra"]) * 100
pivot_table

,ID_Muestra,Porcentaje
Clase,,
2,458,65.522175
4,241,34.477825
Total,699,100.000000


In [5]:
#de los 699 guardaremos el 60% para entrenamiento
train_df, temporal = train_test_split(df, test_size=0.4, random_state=0)  
#en temporal quedo guardado el 40% de los datos originales
#asi que si dividimos en dos el 40% nos quedaran dos grupos de 20% cada uno
valid_df, test_df = train_test_split(temporal, test_size=0.5, random_state=0)  # 20% validación, 20% prueba


In [26]:
#Buscamos algun valor nulo 
print(df.isnull().values.any())

#Al no encontrar valores nulos buscamos algun valor no numerico en nuestro dataset
print(df[df.applymap(lambda x: not str(x).replace('.', '', 1).isdigit()).any(axis=1)])


False
     ID_Muestra  Grosor_Tumor  Uniformidad_Tamaño_Celular  \
23      1057013             8                           4   
40      1096800             6                           6   
139     1183246             1                           1   
145     1184840             1                           1   
158     1193683             1                           1   
164     1197510             5                           1   
235     1241232             3                           1   
249      169356             3                           1   
275      432809             3                           1   
292      563649             8                           8   
294      606140             1                           1   
297       61634             5                           4   
315      704168             4                           6   
321      733639             3                           1   
411     1238464             1                           1   
617     1057067   

/var/folders/rl/jjdlbdsd5dzcsz5tp6_835300000gn/T/ipykernel_3939/2782451103.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  print(df[df.applymap(lambda x: not str(x).replace('.', '', 1).isdigit()).any(axis=1)])


In [27]:
#Encontramos 16 valores "?" en la columna "Núcleos_Desnudos"
